In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import math
import random
import csv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from pywt import wavedec

In [5]:
import os
dataset_path = '../Preprocessing/'
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
    for d in dirnames:
      for j, (dirpath2, dirnames2, filenames2) in enumerate(os.walk(os.path.join(dirpath, d))):
        for f in filenames2:
          print(os.path.join(dirpath2, f))

../Preprocessing/db1/data_mfccRMScqtdb1L10.csv
../Preprocessing/db1/data_mfccRMScqtdb1L7.csv
../Preprocessing/db1/data_mfccRMScqtdb1L1.csv
../Preprocessing/db1/data_mfccRMScqtdb1L6.csv
../Preprocessing/db1/data_mfccRMScqtdb1L5.csv
../Preprocessing/db1/data_mfccRMScqtdb1L9.csv
../Preprocessing/db1/data_mfccRMScqtdb1L8.csv
../Preprocessing/db1/data_mfccRMScqtdb1L4.csv
../Preprocessing/db1/data_mfccRMScqtdb1L2.csv
../Preprocessing/db1/data_mfccRMScqtdb1L3.csv
../Preprocessing/db5/data_mfccDWTcqtdb5L2.csv
../Preprocessing/db5/data_mfccDWTcqtdb5L9.csv
../Preprocessing/db5/data_mfccDWTcqtdb5L6.csv
../Preprocessing/db5/data_mfccDWTcqtdb5L7.csv
../Preprocessing/db5/data_mfccDWTcqtdb5L5.csv
../Preprocessing/db5/data_mfccDWTcqtdb5L3.csv
../Preprocessing/db5/data_mfccDWTcqtdb5L4.csv
../Preprocessing/db5/data_mfccDWTcqtdb5L10.csv
../Preprocessing/db5/data_mfccDWTcqtdb5L1.csv
../Preprocessing/db5/data_mfccDWTcqtdb5L8.csv
../Preprocessing/db8/data_mfccDWTcqtdb8L3.csv
../Preprocessing/db8/data_mfccDW

In [6]:
import os
dataset_path = '../Preprocessing/'
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
    for d in dirnames:
      for j, (dirpath2, dirnames2, filenames2) in enumerate(os.walk(os.path.join(dirpath, d))):
        for f in filenames2:
          file_path = os.path.join(dirpath2, f)
          print("Processing file: ", file_path)
          df = pd.read_csv(file_path)

          X = df.drop(['label', 'filename'], axis=1)
          y = df.iloc[:, -1]

          X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

          lb = LabelEncoder()
          X_train = np.expand_dims(X_train, axis=2)
          X_test = np.expand_dims(X_test, axis=2)
          y_test = tf.keras.utils.to_categorical(lb.fit_transform(y_test))
          y_train = tf.keras.utils.to_categorical(lb.fit_transform(y_train))

          model = tf.keras.Sequential([
              tf.keras.layers.LSTM(64, input_shape=(X_train.shape[1],X_train.shape[2]), return_sequences=True),
              tf.keras.layers.Dense(32,activation='relu', activity_regularizer=tf.keras.regularizers.l2(1e-4)),

              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(64, activation='relu', activity_regularizer=tf.keras.regularizers.l2(1e-4)),
              tf.keras.layers.Dense(32, activation='relu', activity_regularizer=tf.keras.regularizers.l2(1e-4)),
              
              tf.keras.layers.BatchNormalization(),
              tf.keras.layers.Dense(5, activation='softmax')
          ])

          optimazer = tf.keras.optimizers.Adam(learning_rate=0.0001)
          model.compile(optimizer=optimazer, loss='categorical_crossentropy', metrics=['acc'])

          history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

          test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

          to_append = f'{file_path}'
          to_append += f' {test_accuracy}'
          file = open('../Preprocessing/hasil.csv', 'a', newline='')
          with file:
              writer = csv.writer(file)
              writer.writerow(to_append.split())

Processing file:  ../Preprocessing/db1/data_mfccRMScqtdb1L10.csv
Epoch 1/20
50/50 [==============================] - 4s 30ms/step - loss: 1.6947 - acc: 0.2725 - val_loss: 1.5906 - val_acc: 0.3100
Epoch 2/20
50/50 [==============================] - 1s 20ms/step - loss: 1.4450 - acc: 0.3825 - val_loss: 1.5789 - val_acc: 0.3600
Epoch 3/20
50/50 [==============================] - 1s 20ms/step - loss: 1.3432 - acc: 0.4288 - val_loss: 1.5659 - val_acc: 0.3850
Epoch 4/20
50/50 [==============================] - 1s 20ms/step - loss: 1.2604 - acc: 0.4900 - val_loss: 1.5525 - val_acc: 0.3550
Epoch 5/20
50/50 [==============================] - 1s 20ms/step - loss: 1.2040 - acc: 0.5150 - val_loss: 1.5352 - val_acc: 0.3800
Epoch 6/20
50/50 [==============================] - 1s 20ms/step - loss: 1.1508 - acc: 0.5487 - val_loss: 1.5137 - val_acc: 0.3850
Epoch 7/20
50/50 [==============================] - 1s 23ms/step - loss: 1.0950 - acc: 0.5875 - val_loss: 1.4972 - val_acc: 0.3800
Epoch 8/20
50/50 [

KeyboardInterrupt: 